Model version V2

In [1]:
# ---------- improved_training.py ----------
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import random

# Load your preprocessed arrays (already grayscale, 75x75)
data = np.load("preprocessed_data/rafdb_preprocessed.npz")
X_train, y_train = data["X_train"], data["y_train"]
X_val, y_val = data["X_val"], data["y_val"]
X_test, y_test = data["X_test"], data["y_test"]

# If y_train are one-hot, convert to integer labels for class_weight calculation
y_train_int = np.argmax(y_train, axis=1)

# compute class weights (balanced)
classes = np.unique(y_train_int)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train_int)
class_weight_dict = {int(c): w for c, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# ---------------------------
# Augmentation with CLAHE/brightness
# ---------------------------
def preprocessing_fn(img):
    # img arrives as float32 in [0,1] by ImageDataGenerator, shape (h,w,channels)
    # convert back to uint8 for cv operations, apply random CLAHE or brightness
    img_uint8 = np.clip(img * 255.0, 0, 255).astype(np.uint8).squeeze()  # remove channel if single
    # Randomly apply one of: none, CLAHE, brighten/darken
    r = random.random()
    if r < 0.25:
        # CLAHE (adaptive equalization)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        out = clahe.apply(img_uint8)
    elif r < 0.5:
        # Brighten / darken
        alpha = random.uniform(0.9, 1.2)  # contrast
        beta = random.randint(-20, 30)    # brightness
        out = cv2.convertScaleAbs(img_uint8, alpha=alpha, beta=beta)
    else:
        out = img_uint8
    # convert back to float [0,1] and keep channel dim
    out = out.astype("float32") / 255.0
    out = np.expand_dims(out, axis=-1)
    return out

# image data generator (we keep geometric aug similar to before but add preprocessing_fn)
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    preprocessing_function=preprocessing_fn
)

batch_size = 64
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)

# Validation generator (no aggressive augmentation, only rescale already applied)
val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size, shuffle=False)

# ---------------------------
# Define model (same arch)
# ---------------------------
def build_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(75,75,1)),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()

# callbacks
os.makedirs("models", exist_ok=True)
checkpoint = ModelCheckpoint("models/emotion_v2.keras", monitor="val_accuracy", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=1)
early = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1)

# Train (more epochs; you can stop earlier via callbacks)
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=val_generator,
    validation_steps=len(X_val) // batch_size,
    epochs=40,
    class_weight=class_weight_dict,
    callbacks=[checkpoint, reduce_lr, early]
)

# Evaluate on test set (best model will be saved)
best = load_model("models/emotion_v2.keras")
test_loss, test_acc = best.evaluate(X_test, y_test, verbose=1)
print("Test accuracy after retrain:", test_acc)
print("Test loss after retrain:", test_loss)



Class weights: {0: np.float64(1.0), 1: np.float64(1.0), 2: np.float64(1.0), 3: np.float64(1.0), 4: np.float64(1.0), 5: np.float64(1.0), 6: np.float64(1.0)}


C:\Users\AKASH J\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 73, 73, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 73, 73, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 34, 34, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 34, 34, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 17, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 15, 15, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 15, 15, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 897,415 (3.42 MB)

...

Epoch 1/40


C:\Users\AKASH J\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 568ms/step - accuracy: 0.2166 - loss: 2.1303
Epoch 1: val_accuracy improved from None to 0.28058, saving model to models/emotion_v2.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 283s 597ms/step - accuracy: 0.2372 - loss: 1.9332 - val_accuracy: 0.2806 - val_loss: 1.8932 - learning_rate: 1.0000e-04
Epoch 2/40
  1/469 ━━━━━━━━━━━━━━━━━━━━ 3:54 501ms/step - accuracy: 0.2969 - loss: 1.8264

C:\Users\AKASH J\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.28058 to 0.28178, saving model to models/emotion_v2.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.2969 - loss: 1.8264 - val_accuracy: 0.2818 - val_loss: 1.8857 - learning_rate: 1.0000e-04
Epoch 3/40
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 550ms/step - accuracy: 0.2958 - loss: 1.7842
Epoch 3: val_accuracy improved from 0.28178 to 0.44458, saving model to models/emotion_v2.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 292s 580ms/step - accuracy: 0.3003 - loss: 1.7692 - val_accuracy: 0.4446 - val_loss: 1.4914 - learning_rate: 1.0000e-04
Epoch 4/40
  1/469 ━━━━━━━━━━━━━━━━━━━━ 3:43 477ms/step - accuracy: 0.4375 - loss: 1.5676
Epoch 4: val_accuracy did not improve from 0.44458
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 47ms/step - accuracy: 0.4375 - loss: 1.5676 - val_accuracy: 0.4438 - val_loss: 1.4901 - learning_rate: 1.0000e-04
Epoch 5/40
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.3412 - loss: 1.6963
Epoch 5: val_accuracy improved from 0.44458 to 

In [7]:

best_model = load_model("models/emotion_v2.keras")

test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=1)

print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.7179 - loss: 0.7696

✅ Test Accuracy: 0.7179
Test Loss: 0.7696


In [8]:
best_model = load_model("models/emotion_v2.keras")

test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=1)

print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.7179 - loss: 0.7696

✅ Test Accuracy: 0.7179
Test Loss: 0.7696
